Plantilla de Notebook para K-Means Clustering
1. Configuración e Importación de Librerías
Esta sección establece el entorno y carga las librerías necesarias para la manipulación de datos, visualización y el algoritmo K-Means.
## Importación de librerías esenciales
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler # Aunque las fuentes usan Min-Max, StandardScaler es una alternativa común.

from sklearn.decomposition import PCA # Utilizado en algunos contextos de ML, mencionado en las librerías importadas.



Citas:
2. Carga y Exploración Inicial de Datos
Se carga el conjunto de datos (bdclientes.csv) y se realiza una exploración inicial para comprender su estructura y contenido.
## Carga de datos
df = pd.read_csv('bdclientes.csv')

## Exploración de las primeras filas
print(df.head())
## Columnas observadas en el ejemplo: IDCLIENTE, Genero, Tiene_auto, Tiene_propiedad, Num_hijos, Ingreso_anual, Categoria_ingresos, Estudios, Estado_civil [1]

# Exploración estadística (ejemplo basado en las fuentes)
print(df.describe())
## Muestra estadísticas para variables numéricas como IDCLIENTE, Tiene_auto, Tiene_propiedad, Num_hijos, Ingreso_anual [3, 4]
Citas:
3. Preparación y Limpieza de Datos
Esta fase es crucial para asegurar que las variables estén en un formato adecuado para el algoritmo K-Means (numérico y escalado).
3.1. Limpieza y Unificación de Variables Categóricas
Se unifican categorías y se realiza una limpieza inicial.
## Unificación de 'Estado_civil' (Civil marriage se combina con Married)
df.Estado_civil = df.Estado_civil.replace({'Civil marriage' : 'Married'})
## Se observa que el conteo de 'Estado_civil' cambia: Married (261593), Single/not married (42730), Separated (20973), Widow (15177) [5, 6]
Citas:
3.2. Eliminación de Variables Innecesarias
Se eliminan variables que no se usarán para el clustering o que son identificadores.
## Eliminación de la variable 'Estudios' para efectos del ejercicio [6]
df = df.drop('Estudios', axis=1)

## Eliminación de la columna de identificación 'IDCLIENTE' [7]
df_dummies =df_dummies.drop('IDCLIENTE', axis=1) # Asumiendo que esta operación se realiza sobre el DataFrame preparado.
Citas:
3.3. Codificación de Variables Categóricas
Se convierten las variables categóricas a formato numérico (codificación binaria y One-Hot Encoding).
# Codificación binaria de 'Genero' (F: 0, M: 1) [6]
df.Genero = df.Genero.replace({'F' : 0, 'M' : 1})

## Codificación One-Hot para variables restantes
df_dummies = pd.get_dummies(df, columns=['Categoria_ingresos', 'Estado_civil'])
## Esto crea nuevas columnas dummy para cada categoría [6, 7]

## Convertir valores booleanos (True/False) resultantes de get_dummies a 1 y 0 [8, 9]
df_dummies = df_dummies.replace({True: 1, False: 0})
Citas:
3.4. Normalización de Datos
Se aplica la Normalización Min-Max para asegurar que ninguna variable domine el análisis debido a su escala.
## Manejar posibles valores nulos antes de la normalización
df_dummies.dropna(inplace=True)

## Normalización Min-Max
df_norm = (df_dummies - df_dummies.min()) / (df_dummies.max() - df_dummies.min())
## df_norm ahora contiene los datos escalados entre 0 y 1 [10]
Citas:
4. Determinación del Número Óptimo de Clusters (Método del Codo)
Se utiliza el Método del Codo (Elbow Method) analizando la Suma de Cuadrados Dentro del Cluster (WCSS) para encontrar el número $k$ ideal.
wcs =[]

## Iterar de 1 a 15 clusters
for i in range(1,16):
    kmeans = KMeans(n_clusters=i, random_state=0, n_init='auto') # n_init='auto' para versiones recientes de sklearn
    kmeans.fit(df_norm)
    wcs.append(kmeans.inertia_)

## Visualización del Método del Codo
plt.figure(figsize=(8, 5))
plt.grid()
plt.plot(range(1,16), wcs, marker='+', color='blue')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.title('Método del codo para determinar el número óptimo de clusters')
plt.show()
## Basado en el análisis gráfico, las fuentes sugieren que 6 clusters es el número óptimo [11, 12]
Citas:
5. Modelado K-Means y Cálculo de Centroides
Se aplica el algoritmo K-Means utilizando el número óptimo de clusters (en este caso, $k=6$) y se capturan los centroides.
## Aplicación del algoritmo K-Means con k=6
num_clusters = 6
clustering = KMeans(n_clusters=num_clusters, max_iter=300, random_state=0, n_init='auto')
clustering.fit(df_norm)

## Capturar los centroides del modelo
center = clustering.cluster_centers_
## Los centroides están en formato normalizado [12]
Citas:
6. Interpretación de Resultados (Desnormalización)
Para interpretar los perfiles de los clientes, los centroides deben ser llevados a su escala original.
6.1. Función de Desnormalización
Se define una función que revierte la normalización Min-Max.
def desnormalizar(df, columns_name_list, num_clusters):
    """Revierte la normalización Min-Max aplicada a los centroides."""
    data_desnorm = [[] for _ in range(num_clusters)]
    for i in range(num_clusters):
        for j in range(len(columns_name_list)):
            # Fórmula de desnormalización: Min + Centroide_Normalizado * (Max - Min)
            data_desnorm[i].append(df[columns_name_list[j]].min() + center[i][j]*(df[columns_name_list[j]].max() - df[columns_name_list[j]].min()))
    return np.asarray(data_desnorm)
Citas:
6.2. Aplicación y Organización de Centroides
Se aplica la desnormalización y se organizan los resultados en un DataFrame legible.
## Nombres de las columnas originales
columns_name = list(df_norm.columns.values)

## Desnormalizar los centroides
df_desnorm = desnormalizar(df_dummies, columns_name, num_clusters)

## Organizar los resultados en un DataFrame con nombres de clusters
names = []
for i in range(num_clusters):
    names.append('Cluster_{}' .format(i+1))

df_centroides = pd.DataFrame(df_desnorm.transpose(), columns=names).abs().round(3)
columns_names_series = pd.Series(df_norm.columns.values)
df_centroides.set_index([columns_names_series], inplace=True)

## Visualizar los centroides desnormalizados
df_centroides